In [1]:
import numpy as np
import pandas as pd
import random

from pathlib import Path
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, GRU, Input, BatchNormalization, Dropout
from ncps.wirings import AutoNCP
from ncps.keras import LTC

2025-04-06 23:38:38.643386: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-06 23:38:38.650483: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-06 23:38:38.677045: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743971918.721248    3611 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743971918.732025    3611 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 23:38:38.777378: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Configuration

In [2]:
NUM_EPOCHS = 200
NUM_EXPERIMENTS = 5

def create_model(train):
    model = Sequential()
    model.add(Input(shape=(train.shape[1], train.shape[2])))

    model.add(Dropout(0.2))
    model.add(LTC(20, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(20, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=True))

    model.add(Dropout(0.2))
    model.add(LTC(10, return_sequences=False))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.002, weight_decay=1e-7, use_ema=True), loss='binary_crossentropy', metrics=["accuracy", AUC(name="auc")])
    return model

# Experiment

In [3]:
ID = ["ID"]
USER = ["SubjectID"]
IDS = ["SubjectID", "VideoID"]
TARGET = ["predefinedlabel"]
FEATURES = ["Delta", "Theta", "Alpha1", "Alpha2", "Beta1", "Beta2", "Gamma1", "Gamma2"]
LAGS = [1, 2]
INIT_SEED = 5412

In [4]:
data_dir = Path("/home/aseliverstov/projects/brain_signals/data")
data = pd.read_csv(data_dir / "EEG_data.csv")

new_features = []
for lag in LAGS:
    for feature_name in FEATURES:
        new_feature_name = f"{feature_name}_{lag}"
        new_features.append(new_feature_name)
        data[new_feature_name] = data.groupby(IDS)[feature_name].shift(lag).fillna(0)
FEATURES.extend(new_features)

data["ID"] = (len(np.unique(data["VideoID"])) * data["SubjectID"] + data["VideoID"]).astype("int")
data = data[ID + USER + FEATURES + TARGET]

data.head(3)

,ID,SubjectID,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,...,Gamma2_1,Delta_2,Theta_2,Alpha1_2,Alpha2_2,Beta1_2,Beta2_2,Gamma1_2,Gamma2_2,predefinedlabel
0,0,0.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,...,8293.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,...,2740.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0


In [5]:
def reshape_dataset(data):
    features = []
    target = []
    for cur_id in np.unique(data[ID].to_numpy()):
        cur_id_data = data[data[ID].to_numpy() == cur_id]
        target.append(np.mean(cur_id_data[TARGET].to_numpy()).astype("int"))
        features.append(cur_id_data[FEATURES].to_numpy())

    features = pad_sequences(features)
    return np.array(features), np.array(target)

def pad_sequences(arrays, pad_value=0):
    max_length = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        np.pad(
            arr,
            ((0, max_length - arr.shape[0]), (0, 0)),
            mode='constant',
            constant_values=pad_value)
            for arr in arrays
        ]
    return np.stack(padded_arrays)

In [6]:
X, _ = reshape_dataset(data)
model = create_model(X)
model.summary()

2025-04-06 23:38:44.834733: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 144, 24)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc (LTC)                       │ (None, 144, 20)        │         3,668 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 144, 20)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_1 (LTC)                     │ (None, 144, 20)        │         3,340 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 144, 20)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_2 (LTC)                     │ (None, 144, 10)        │         1,290 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 144, 10)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ ltc_3 (LTC)                     │ (None, 10)             │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,179 (35.86 KB)

 Trainable params: 9,179 (35.86 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
%%time

all_acc = []
all_loss = []
all_auc = []

all_val_acc = []
all_val_loss = []
all_val_auc = []

for j, seed in tqdm(enumerate(np.arange(NUM_EXPERIMENTS) + INIT_SEED)):
    np.random.seed(int(seed))
    random.seed(int(seed))
    tf.random.set_seed(int(seed))

    train_id = np.random.choice(np.unique(np.ravel(data[USER])), 7, replace=False)
    train_index = np.isin(data[USER], train_id)

    train = data.iloc[train_index]
    test = data.iloc[~train_index]

    X_train, y_train = reshape_dataset(train)
    X_test, y_test = reshape_dataset(test)

    y_train = y_train.reshape(-1, 1)
    y_test = y_test.reshape(-1, 1)

    model = create_model(X_train)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=NUM_EPOCHS,
        batch_size=10,
        verbose=1,
    )

    acc = history.history['accuracy']
    loss = history.history['loss']
    auc = history.history['auc']

    val_acc = history.history['val_accuracy']
    val_loss = history.history['val_loss']
    val_auc = history.history['val_auc']

    all_acc.append(acc)
    all_loss.append(loss)
    all_auc.append(auc)

    all_val_acc.append(val_acc)
    all_val_loss.append(val_loss)
    all_val_auc.append(val_auc)

epoch_acc = np.mean(all_acc, axis=0)
epoch_loss = np.mean(all_loss, axis=0)
epoch_auc = np.mean(all_auc, axis=0)

epoch_val_acc = np.mean(all_val_acc, axis=0)
epoch_val_loss = np.mean(all_val_loss, axis=0)
epoch_val_auc = np.mean(all_val_auc, axis=0)

0it [00:00, ?it/s]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 63s 3s/step - accuracy: 0.4665 - auc: 0.5504 - loss: 0.7562 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6995
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.4496 - auc: 0.4557 - loss: 0.7086 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6916
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5429 - auc: 0.5440 - loss: 0.6888 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6910
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5179 - auc: 0.4528 - loss: 0.6969 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6902
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.4471 - auc: 0.4870 - loss: 0.6955 - val_accuracy: 0.6333 - val_auc: 0.6333 - val_loss: 0.6889
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5692 - auc: 0.5164 - loss: 0.6934 - val_accuracy: 0.6333 - val_auc: 0.6333 - val_loss: 0.6881
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accurac

1it [57:08, 3428.42s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 84s 3s/step - accuracy: 0.4364 - auc: 0.3941 - loss: 0.7143 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.6933
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 16s 2s/step - accuracy: 0.4912 - auc: 0.4807 - loss: 0.6946 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6930
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5692 - auc: 0.5588 - loss: 0.6900 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6924
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5948 - auc: 0.6118 - loss: 0.6868 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6919
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.4946 - auc: 0.4598 - loss: 0.6969 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6912
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.5569 - auc: 0.6863 - loss: 0.6836 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6900
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accurac

2it [1:55:23, 3467.81s/it]

...

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.5113 - auc: 0.4787 - loss: 0.8494 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.7555
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.5113 - auc: 0.4943 - loss: 0.7430 - val_accuracy: 0.5000 - val_auc: 0.5000 - val_loss: 0.7028
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.5113 - auc: 0.4786 - loss: 0.7117 - val_accuracy: 0.6333 - val_auc: 0.6333 - val_loss: 0.6918
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.4923 - auc: 0.4317 - loss: 0.7056 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6961
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.5171 - auc: 0.4592 - loss: 0.6951 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6983
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.5423 - auc: 0.5775 - loss: 0.6881 - val_accuracy: 0.5000 - val_auc: 0.6333 - val_loss: 0.6957
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accurac

4it [3:54:06, 3520.74s/it]

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 90s 3s/step - accuracy: 0.3571 - auc: 0.3519 - loss: 0.7283 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6967
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.4421 - auc: 0.4569 - loss: 0.7017 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6936
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.6839 - auc: 0.5978 - loss: 0.6786 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6920
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.5459 - auc: 0.6024 - loss: 0.6854 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6925
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.4738 - auc: 0.4203 - loss: 0.7186 - val_accuracy: 0.5000 - val_auc: 0.6000 - val_loss: 0.6917
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.6451 - auc: 0.6211 - loss: 0.6850 - val_accuracy: 0.5000 - val_auc: 0.7000 - val_loss: 0.6903
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accurac

5it [4:44:51, 3418.31s/it]

CPU times: user 8h 7min 19s, sys: 2h 43s, total: 10h 8min 2s
Wall time: 4h 44min 51s


In [8]:
for i in range(NUM_EPOCHS):
    print(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}")
    print(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}")

with open("./logs/Liquidv3_emb2.txt", "w") as f:
    for i in range(NUM_EPOCHS):
        f.write(f"Epoch {(i + 1)}: TRAIN Accuracy = {np.round(epoch_acc[i], 3)} Loss = {np.round(epoch_loss[i], 3)} AUC = {np.round(epoch_auc[i], 3)}\n")
        f.write(f"Epoch {(i + 1)}: VAL Accuracy = {np.round(epoch_val_acc[i], 3)} Loss = {np.round(epoch_val_loss[i], 3)} AUC = {np.round(epoch_val_auc[i], 3)}\n")

Epoch 1: TRAIN Accuracy = 0.469 Loss = 0.754 AUC = 0.473
Epoch 1: VAL Accuracy = 0.5 Loss = 0.717 AUC = 0.573
Epoch 2: TRAIN Accuracy = 0.483 Loss = 0.712 AUC = 0.486
Epoch 2: VAL Accuracy = 0.5 Loss = 0.697 AUC = 0.567
Epoch 3: TRAIN Accuracy = 0.543 Loss = 0.693 AUC = 0.532
Epoch 3: VAL Accuracy = 0.527 Loss = 0.692 AUC = 0.593
Epoch 4: TRAIN Accuracy = 0.534 Loss = 0.69 AUC = 0.557
Epoch 4: VAL Accuracy = 0.5 Loss = 0.693 AUC = 0.593
Epoch 5: TRAIN Accuracy = 0.486 Loss = 0.698 AUC = 0.489
Epoch 5: VAL Accuracy = 0.527 Loss = 0.693 AUC = 0.593
Epoch 6: TRAIN Accuracy = 0.569 Loss = 0.691 AUC = 0.558
Epoch 6: VAL Accuracy = 0.527 Loss = 0.692 AUC = 0.613
Epoch 7: TRAIN Accuracy = 0.546 Loss = 0.691 AUC = 0.536
Epoch 7: VAL Accuracy = 0.52 Loss = 0.69 AUC = 0.613
Epoch 8: TRAIN Accuracy = 0.526 Loss = 0.689 AUC = 0.557
Epoch 8: VAL Accuracy = 0.5 Loss = 0.689 AUC = 0.593
Epoch 9: TRAIN Accuracy = 0.534 Loss = 0.689 AUC = 0.566
Epoch 9: VAL Accuracy = 0.567 Loss = 0.687 AUC = 0.62
Epoc